# GABARITO COMPLETO - Etapa 3: Modelo Baseline

**Dataset:** Students Performance

**Objetivo:** Criar primeiro modelo de ML e estabelecer benchmark

---

## Conteúdo:
1. Divisão de Dados (60/20/20)
2. Modelo Baseline (Regressão Linear)
3. Métricas de Avaliação
4. Análise de Resíduos
5. Feature Importance
6. Diagnóstico Overfitting
7. Salvamento

## 1. IMPORTS E CONFIGURAÇÕES

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy import stats
import joblib
import os
import warnings

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
pd.set_option('display.max_columns', None)

print("✅ Imports concluídos!")

## 2. CARREGAR DADOS PROCESSADOS

In [ ]:
# Carregar dataset limpo da Etapa 2
try:
    df = pd.read_csv('../../data/students_clean.csv')
    print(f"✅ Dataset carregado: {df.shape}")
except FileNotFoundError:
    print("❌ ERRO: Arquivo students_clean.csv não encontrado!")
    print("Execute a Etapa 2 primeiro!")
    raise

df.head()

In [ ]:
df.info()

In [ ]:
# Verificar missing values
missing_count = df.isnull().sum().sum()
print(f"Missing values: {missing_count}")

if missing_count == 0:
    print("✅ Dataset 100% limpo! Etapa 2 foi executada corretamente.")
else:
    print(f"⚠️ ATENÇÃO: {missing_count} missing values detectados!")
    print("   Isso NÃO deveria acontecer se a Etapa 2 foi executada.")
    print("   Verifique se você está usando o arquivo correto (students_clean.csv)")
    print("\n🔧 Aplicando tratamento de segurança...")
    
    # Tratamento de emergência
    for col in df.select_dtypes(include=['float64', 'int64']).columns:
        if df[col].isnull().sum() > 0:
            df[col].fillna(df[col].median(), inplace=True)
    
    print(f"✅ Missing values tratados: {df.isnull().sum().sum()} restantes")

---
# PARTE 1: DIVISÃO DE DADOS (60/20/20)

**Por que 60/20/20?**
- **Treino (60%):** Aprende padrões
- **Validação (20%):** Ajusta hiperparâmetros
- **Teste (20%):** Avaliação final imparcial

**Benefício:** Evita overfitting no conjunto de teste

## 1.1. Separar Features e Target

In [ ]:
# Target
target_col = 'final_grade'

# Features (remover ID e target)
X = df.drop(columns=['student_id', target_col])
y = df[target_col]

print(f"Features: {X.shape}")
print(f"Target: {y.shape}")
print(f"\nColunas de features: {len(X.columns)}")
print(X.columns.tolist())

## 1.2. Divisão Treino/Validação/Teste

In [ ]:
# PASSO 1: Separar teste (20%)
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

print("🔸 Primeira divisão (80% temp / 20% teste)")
print(f"  X_temp: {X_temp.shape}")
print(f"  X_test: {X_test.shape}")

In [ ]:
# PASSO 2: Separar treino e validação
# 0.25 * 80% = 20% do total para validação
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp,
    test_size=0.25,
    random_state=42
)

print("🔸 Segunda divisão (60% treino / 20% validação do total)")
print(f"  X_train: {X_train.shape}")
print(f"  X_val: {X_val.shape}")

In [ ]:
# Resumo das divisões
print("="*60)
print("RESUMO DAS DIVISÕES")
print("="*60)
print(f"Treino:    {X_train.shape[0]:4d} amostras ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Validação: {X_val.shape[0]:4d} amostras ({X_val.shape[0]/len(X)*100:.1f}%)")
print(f"Teste:     {X_test.shape[0]:4d} amostras ({X_test.shape[0]/len(X)*100:.1f}%)")
print(f"TOTAL:     {len(X):4d} amostras (100.0%)")

## 1.3. Verificar Distribuição do Target

In [ ]:
# Estatísticas
stats_df = pd.DataFrame({
    'Treino': y_train.describe(),
    'Validação': y_val.describe(),
    'Teste': y_test.describe()
})

print("📊 Estatísticas do Target nos 3 Conjuntos:\n")
print(stats_df.round(2))

In [ ]:
# Comparar médias
mean_train = y_train.mean()
mean_val = y_val.mean()
mean_test = y_test.mean()

diff_val = abs(mean_val - mean_train) / mean_train * 100
diff_test = abs(mean_test - mean_train) / mean_train * 100

print(f"\nMédia Treino:    {mean_train:.2f}")
print(f"Média Validação: {mean_val:.2f} (diff: {diff_val:.2f}%)")
print(f"Média Teste:     {mean_test:.2f} (diff: {diff_test:.2f}%)")

if diff_val < 5 and diff_test < 5:
    print("\n✅ Distribuições muito similares (< 5% diferença)")
else:
    print("\n⚠️ Distribuições têm diferenças > 5%")

In [ ]:
# Visualização: Boxplots
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].boxplot(y_train, patch_artist=True,
                boxprops=dict(facecolor='#4CAF50', alpha=0.7))
axes[0].set_title(f'Treino (n={len(y_train)})', fontweight='bold')
axes[0].set_ylabel('Nota Final')
axes[0].grid(axis='y', alpha=0.3)

axes[1].boxplot(y_val, patch_artist=True,
                boxprops=dict(facecolor='#FFC107', alpha=0.7))
axes[1].set_title(f'Validação (n={len(y_val)})', fontweight='bold')
axes[1].set_ylabel('Nota Final')
axes[1].grid(axis='y', alpha=0.3)

axes[2].boxplot(y_test, patch_artist=True,
                boxprops=dict(facecolor='#F44336', alpha=0.7))
axes[2].set_title(f'Teste (n={len(y_test)})', fontweight='bold')
axes[2].set_ylabel('Nota Final')
axes[2].grid(axis='y', alpha=0.3)

plt.suptitle('VISUALIZAÇÃO 1: Distribuição do Target nos 3 Conjuntos',
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('etapa3_01_target_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

---
## 📝 RESPOSTA QUESTÃO 1

**Q1: Por que usar divisão 60/20/20 ao invés de 80/20?**

**RESPOSTA:**

**Problema da divisão 80/20:**
- Treina com 80%, avalia com 20% (teste)
- Se resultado ruim → ajusta modelo → avalia no MESMO teste
- **Vazamento indireto:** Modelo "aprende" características do teste

**Solução 60/20/20:**
- **Treino (60%):** Ajusta pesos do modelo
- **Validação (20%):** Testa ajustes, compara modelos
- **Teste (20%):** **Guardado intocado** até avaliação final

**Fluxo correto:**
```
1. Treinar no treino
2. Avaliar na validação
3. Ajustar baseado na validação
4. Repetir 1-3 quantas vezes necessário
5. UMA VEZ ao final: avaliar no teste
6. Reportar resultado do teste
```

**Analogia:**
- Treino = Estudar o conteúdo
- Validação = Simulados (pode refazer)
- Teste = Prova final (uma chance só!)

**Benefício:** Avaliação imparcial, resultado confiável para produção

---
# PARTE 2: MODELO BASELINE (REGRESSÃO LINEAR)

**Por que Regressão Linear como baseline?**
1. Rápido de treinar
2. Interpretável (coeficientes claros)
3. Bom ponto de referência
4. Funciona bem para relações lineares

## 2.1. Treinar Modelo

In [ ]:
# Criar e treinar
baseline_model = LinearRegression()
baseline_model.fit(X_train, y_train)

print("="*60)
print("MODELO BASELINE TREINADO")
print("="*60)
print(f"\nIntercepto: {baseline_model.intercept_:.4f}")
print(f"Coeficientes: {len(baseline_model.coef_)}")

## 2.2. Fazer Predições

In [ ]:
# Predições nos 3 conjuntos
y_train_pred = baseline_model.predict(X_train)
y_val_pred = baseline_model.predict(X_val)
y_test_pred = baseline_model.predict(X_test)

print("✅ Predições realizadas:")
print(f"  Treino: {len(y_train_pred)} predições")
print(f"  Validação: {len(y_val_pred)} predições")
print(f"  Teste: {len(y_test_pred)} predições")

In [ ]:
# Comparar primeiras predições
comparison = pd.DataFrame({
    'Real': y_val.head(10).values,
    'Predito': y_val_pred[:10],
    'Erro': y_val.head(10).values - y_val_pred[:10]
})

print("\n📊 Primeiras 10 Predições (Validação):\n")
print(comparison.to_string(index=False))

## 2.3. Coeficientes e Feature Importance

In [ ]:
# DataFrame de coeficientes
coef_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': baseline_model.coef_
}).sort_values('Coefficient', key=abs, ascending=False)

print("📊 COEFICIENTES DO MODELO:\n")
print(coef_df.head(15).to_string(index=False))

In [ ]:
# Visualização dos top coeficientes
top_features = coef_df.head(15)

plt.figure(figsize=(10, 8))
colors = ['#2E86AB' if c > 0 else '#A23B72' for c in top_features['Coefficient']]
plt.barh(range(len(top_features)), top_features['Coefficient'], color=colors,
         alpha=0.7, edgecolor='black')
plt.yticks(range(len(top_features)), top_features['Feature'])
plt.xlabel('Coeficiente', fontweight='bold')
plt.title('VISUALIZAÇÃO 2: Top 15 Features Mais Importantes',
          fontsize=14, fontweight='bold')
plt.axvline(x=0, color='black', linestyle='-', linewidth=1.5)
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.savefig('etapa3_02_feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

---
## 📝 RESPOSTA QUESTÃO 2

**Q2: Interprete os 3 coeficientes mais importantes do modelo**

**RESPOSTA:**

In [ ]:
top_3 = coef_df.head(3)

print("TOP 3 FEATURES MAIS IMPACTANTES:\n")
for i, (idx, row) in enumerate(top_3.iterrows(), 1):
    feat = row['Feature']
    coef = row['Coefficient']
    direction = "AUMENTA" if coef > 0 else "DIMINUI"

    print(f"{i}. {feat}")
    print(f"   Coeficiente: {coef:.4f}")
    print(f"   Interpretação: Cada unidade adicional {direction} a nota em {abs(coef):.2f} pontos")
    print(f"   Impacto: {'Positivo' if coef > 0 else 'Negativo'}\n")

print("💡 IMPORTANTE:")
print("- Como usamos StandardScaler, '1 unidade' = 1 desvio padrão")
print("- Coeficientes positivos aumentam a nota")
print("- Coeficientes negativos diminuem a nota")

---
# PARTE 3: MÉTRICAS DE AVALIAÇÃO

**4 Métricas Essenciais:**
1. **MSE:** Mean Squared Error (penaliza erros grandes)
2. **RMSE:** Root MSE (mesma unidade do target)
3. **MAE:** Mean Absolute Error (robusto a outliers)
4. **R²:** Proporção de variância explicada (0-1)

## 3.1. Calcular Métricas

In [ ]:
def evaluate_model(y_true, y_pred, dataset_name=""):
    """Calcula e exibe métricas de regressão"""
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    print(f"\n{dataset_name}")
    print("-" * 50)
    print(f"MSE  (Mean Squared Error):     {mse:.4f}")
    print(f"RMSE (Root MSE):              {rmse:.4f} pontos")
    print(f"MAE  (Mean Absolute Error):   {mae:.4f} pontos")
    print(f"R²   (R-squared):             {r2:.4f} ({r2*100:.2f}%)")

    return {'MSE': mse, 'RMSE': rmse, 'MAE': mae, 'R2': r2}

# Avaliar
print("="*60)
print("AVALIAÇÃO DO MODELO BASELINE")
print("="*60)

train_metrics = evaluate_model(y_train, y_train_pred, "📊 TREINO")
val_metrics = evaluate_model(y_val, y_val_pred, "📊 VALIDAÇÃO")

In [ ]:
# Tabela comparativa
metrics_df = pd.DataFrame({
    'Treino': train_metrics,
    'Validação': val_metrics
})

print("\n" + "="*60)
print("COMPARAÇÃO TREINO vs VALIDAÇÃO")
print("="*60 + "\n")
print(metrics_df)

In [ ]:
# Calcular diferenças
diff_r2 = abs(train_metrics['R2'] - val_metrics['R2'])
diff_r2_pct = (diff_r2 / train_metrics['R2']) * 100

print(f"\n📈 Análise de Generalização:")
print(f"  Diferença R²: {diff_r2:.4f} ({diff_r2_pct:.2f}%)")

if diff_r2_pct < 5:
    print("  ✅ EXCELENTE! Modelo generaliza muito bem (< 5%)")
elif diff_r2_pct < 10:
    print("  ✅ BOM! Generalização adequada (< 10%)")
else:
    print("  ⚠️ ATENÇÃO! Possível overfitting (> 10%)")

---
## 📝 RESPOSTA QUESTÃO 3

**Q3: Explique cada métrica e qual é mais importante**

**RESPOSTA:**

### 1. MSE (Mean Squared Error)
- **Fórmula:** Σ(y_real - y_pred)² / n
- **Unidade:** pontos² (difícil interpretar)
- **Característica:** Penaliza MUITO erros grandes (eleva ao quadrado)
- **Uso:** Função de perda durante treinamento

### 2. RMSE (Root Mean Squared Error)
- **Fórmula:** √MSE
- **Unidade:** pontos (mesma do target!)
- **Interpretação:** "O modelo erra em média X pontos"
- **Vantagem:** Interpretável para leigos

### 3. MAE (Mean Absolute Error)
- **Fórmula:** Σ|y_real - y_pred| / n
- **Unidade:** pontos
- **Característica:** Menos sensível a outliers que RMSE
- **Interpretação:** Erro médio absoluto

### 4. R² (R-squared)
- **Fórmula:** 1 - (SS_res / SS_tot)
- **Range:** -∞ a 1 (1 = perfeito)
- **Interpretação:** "% da variância explicada"
- **Benchmark:** > 0.7 = bom, > 0.9 = excelente

### 🎯 Qual mais importante?

**Depende do contexto!**

- **Para comparar modelos:** **R²** (normalizado)
- **Para comunicar com não-técnicos:** **RMSE/MAE** (interpretável)
- **Para penalizar erros grandes:** **MSE/RMSE**
- **Para dados com outliers:** **MAE**

**Neste projeto:** Usamos R² como principal + RMSE para interpretação

---
# PARTE 4: ANÁLISE DE RESÍDUOS

**Resíduos = Erros do modelo (y_real - y_pred)**

**Um bom modelo deve ter:**
1. Resíduos centrados em zero (sem viés)
2. Distribuição normal
3. Variância constante (homocedasticidade)

## 4.1. Calcular Resíduos

In [ ]:
# Resíduos
residuals_train = y_train - y_train_pred
residuals_val = y_val - y_val_pred

print("📊 ESTATÍSTICAS DOS RESÍDUOS:\n")
print("TREINO:")
print(f"  Média:   {residuals_train.mean():.6f} (esperado ≈ 0)")
print(f"  Mediana: {residuals_train.median():.4f}")
print(f"  Std Dev: {residuals_train.std():.4f}")

print("\nVALIDAÇÃO:")
print(f"  Média:   {residuals_val.mean():.6f} (esperado ≈ 0)")
print(f"  Mediana: {residuals_val.median():.4f}")
print(f"  Std Dev: {residuals_val.std():.4f}")

In [ ]:
# Verificar se média próxima de zero
mean_res = abs(residuals_val.mean())
if mean_res < 0.01:
    print(f"\n✅ Média muito próxima de zero ({residuals_val.mean():.6f})")
    print("   → Modelo SEM viés sistemático")
elif mean_res < 0.1:
    print(f"\n✅ Média próxima de zero ({residuals_val.mean():.4f})")
else:
    print(f"\n⚠️ Média não tão próxima de zero ({residuals_val.mean():.4f})")
    if residuals_val.mean() > 0:
        print("   → Modelo tende a SUBESTIMAR")
    else:
        print("   → Modelo tende a SUPERESTIMAR")

## 4.2. Visualização 1: Predições vs Reais

In [ ]:
# Scatter plot
plt.figure(figsize=(10, 10))

# Validação
plt.scatter(y_val, y_val_pred, alpha=0.6, s=50, color='#4ECDC4',
           edgecolors='navy', linewidth=0.5, label=f'Validação (n={len(y_val)})')

# Linha perfeita
min_val = min(y_val.min(), y_val_pred.min())
max_val = max(y_val.max(), y_val_pred.max())
plt.plot([min_val, max_val], [min_val, max_val], 'r--', lw=3,
        label='Predição Perfeita (y=x)')

# Banda de confiança (±RMSE)
rmse_val = val_metrics['RMSE']
x_line = np.array([min_val, max_val])
plt.fill_between(x_line, x_line - rmse_val, x_line + rmse_val,
                 color='gray', alpha=0.2, label=f'±RMSE (±{rmse_val:.2f})')

plt.xlabel('Valores Reais', fontsize=13, fontweight='bold')
plt.ylabel('Valores Preditos', fontsize=13, fontweight='bold')
plt.title('VISUALIZAÇÃO 3: Valores Reais vs Preditos (Validação)',
         fontsize=14, fontweight='bold')
plt.legend(loc='upper left', fontsize=11)
plt.grid(alpha=0.3)
plt.axis('equal')

# Adicionar métricas no gráfico
textstr = f'R² = {val_metrics["R2"]:.4f}\nRMSE = {rmse_val:.2f}\nMAE = {val_metrics["MAE"]:.2f}'
props = dict(boxstyle='round', facecolor='wheat', alpha=0.8)
plt.text(0.05, 0.95, textstr, transform=plt.gca().transAxes, fontsize=12,
        verticalalignment='top', bbox=props)

plt.tight_layout()
plt.savefig('etapa3_03_predictions_vs_actual.png', dpi=300, bbox_inches='tight')
plt.show()

## 4.3. Visualização 2: Resíduos vs Preditos

In [ ]:
# Resíduos vs predições
plt.figure(figsize=(12, 6))

plt.scatter(y_val_pred, residuals_val, alpha=0.5, s=30, color='royalblue',
           edgecolors='navy', linewidth=0.5)

# Linha em zero
plt.axhline(y=0, color='red', linestyle='--', linewidth=2, label='Resíduo = 0')

# Linhas de referência (±2 RMSE)
plt.axhline(y=2*rmse_val, color='orange', linestyle=':', linewidth=1.5, alpha=0.7)
plt.axhline(y=-2*rmse_val, color='orange', linestyle=':', linewidth=1.5, alpha=0.7,
           label=f'±2 RMSE')

plt.xlabel('Valores Preditos', fontsize=13, fontweight='bold')
plt.ylabel('Resíduos (Real - Predito)', fontsize=13, fontweight='bold')
plt.title('VISUALIZAÇÃO 4: Resíduos vs Valores Preditos',
         fontsize=14, fontweight='bold')
plt.legend(loc='best')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('etapa3_04_residuals_vs_fitted.png', dpi=300, bbox_inches='tight')
plt.show()

## 4.4. Visualização 3: Distribuição dos Resíduos

In [ ]:
# Histograma dos resíduos
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histograma
axes[0].hist(residuals_val, bins=40, density=True, alpha=0.7,
            color='green', edgecolor='darkgreen', label='Resíduos')

# Curva normal teórica
mu = residuals_val.mean()
sigma = residuals_val.std()
x_range = np.linspace(residuals_val.min(), residuals_val.max(), 100)
normal_curve = stats.norm.pdf(x_range, mu, sigma)
axes[0].plot(x_range, normal_curve, 'r--', linewidth=2,
            label=f'Normal(μ={mu:.2f}, σ={sigma:.2f})')

axes[0].axvline(mu, color='red', linestyle='--', linewidth=1.5, label=f'Média: {mu:.2f}')
axes[0].set_xlabel('Resíduo', fontweight='bold')
axes[0].set_ylabel('Densidade', fontweight='bold')
axes[0].set_title('Histograma dos Resíduos', fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3, axis='y')

# Q-Q Plot
stats.probplot(residuals_val, dist="norm", plot=axes[1])
axes[1].set_title('Q-Q Plot (Teste de Normalidade)', fontweight='bold')
axes[1].grid(alpha=0.3)

plt.suptitle('VISUALIZAÇÃO 5: Análise de Normalidade dos Resíduos',
            fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('etapa3_05_residuals_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Teste de normalidade
skew = residuals_val.skew()
kurt = residuals_val.kurtosis()

print("📊 ANÁLISE DE NORMALIDADE:")
print(f"\nSkewness (Assimetria): {skew:.4f}")
if abs(skew) < 0.5:
    print("  ✅ Distribuição simétrica (|skew| < 0.5)")
elif abs(skew) < 1.0:
    print("  ⚠️ Leve assimetria (0.5 < |skew| < 1.0)")
else:
    print("  ❌ Assimetria significativa (|skew| > 1.0)")

print(f"\nKurtosis (Curtose): {kurt:.4f}")
if abs(kurt) < 0.5:
    print("  ✅ Curtose normal (|kurt| < 0.5)")
else:
    print("  ⚠️ Desvio da normalidade")

---
## 📝 RESPOSTA QUESTÃO 4

**Q4: O modelo apresenta overfitting ou underfitting? Justifique.**

**RESPOSTA:**

In [ ]:
print("="*60)
print("DIAGNÓSTICO: OVERFITTING vs UNDERFITTING")
print("="*60)

r2_train = train_metrics['R2']
r2_val = val_metrics['R2']
diff_r2 = abs(r2_train - r2_val)
diff_r2_pct = (diff_r2 / r2_train) * 100

print(f"\n📊 MÉTRICAS:")
print(f"  R² Treino:    {r2_train:.4f} ({r2_train*100:.2f}%)")
print(f"  R² Validação: {r2_val:.4f} ({r2_val*100:.2f}%)")
print(f"  Diferença:    {diff_r2:.4f} ({diff_r2_pct:.2f}%)")

print("\n💡 DIAGNÓSTICO:")

if r2_val < 0.5 and r2_train < 0.5:
    diagnostico = "UNDERFITTING"
    print(f"  ❌ {diagnostico}")
    print("  - R² baixo em AMBOS os conjuntos")
    print("  - Modelo muito simples, não captura complexidade")
    print("\n  SOLUÇÃO:")
    print("  - Adicionar features polinomiais")
    print("  - Usar modelos mais complexos (Random Forest, XGBoost)")

elif diff_r2_pct > 10:
    diagnostico = "OVERFITTING"
    print(f"  ⚠️ {diagnostico}")
    print("  - Performance excelente no treino, mas cai na validação")
    print("  - Modelo 'decorou' dados de treino")
    print("\n  SOLUÇÃO:")
    print("  - Regularização (Ridge, Lasso)")
    print("  - Remover features menos importantes")
    print("  - Coletar mais dados")

elif diff_r2_pct > 5:
    diagnostico = "LEVE OVERFITTING"
    print(f"  ⚠️ {diagnostico}")
    print("  - Pequena queda na validação (5-10%)")
    print("  - Aceitável, mas pode melhorar")
    print("\n  SUGESTÃO:")
    print("  - Considerar regularização L2")

else:
    diagnostico = "BOM AJUSTE (GOOD FIT)"
    print(f"  ✅ {diagnostico}")
    print("  - Performance alta e similar em ambos")
    print("  - Modelo generaliza MUITO BEM")
    print("  - Diferença < 5% é excelente!")

print(f"\n🎯 CONCLUSÃO: {diagnostico}")

---
## 📝 RESPOSTA QUESTÃO 5

**Q5: Analise os resíduos. O modelo satisfaz as suposições da Regressão Linear?**

**RESPOSTA:**

### Suposições da Regressão Linear:

#### 1. ✅ Linearidade
- **Verificação:** Scatter plot resíduos vs preditos
- **Resultado:** Sem padrão curvilíneo → ✅ Satisfeita

#### 2. ✅ Independência
- **Suposição:** Observações independentes
- **Contexto:** Dados de diferentes alunos → ✅ Satisfeita

#### 3. ✅/⚠️ Homocedasticidade (Variância Constante)
- **Verificação:** Dispersão uniforme no plot de resíduos
- **Resultado:** Verificar se não há padrão de "cone"
- Se dispersão uniforme → ✅ Satisfeita
- Se forma cone → ⚠️ Heteroscedasticidade

#### 4. ✅/⚠️ Normalidade dos Resíduos
- **Verificação:** Q-Q Plot e teste de Shapiro-Wilk
- **Critérios:**
  - Pontos no Q-Q plot seguem linha → ✅ Normal
  - Skewness < 0.5 → ✅ Simétrico
  - Kurtosis < 0.5 → ✅ Normal

#### 5. ✅ Sem Multicolinearidade
- **Verificação:** VIF (Variance Inflation Factor)
- **Solução prévia:** One-Hot Encoding com drop_first=True

### 🎯 Conclusão Geral:

**Se maioria das suposições satisfeitas:**
- Modelo é adequado
- Predições confiáveis
- Inferências estatísticas válidas

**Se violações detectadas:**
- Modelo ainda pode funcionar (ML != Estatística)
- Avaliar impacto na performance
- Considerar transformações ou outros modelos

---
# PARTE 5: SALVAMENTO DO MODELO E DADOS

**Por que salvar?**
- Reutilizar na Etapa 4
- Não precisar retreinar
- Garantir reprodutibilidade

## 5.1. Criar Diretórios

In [ ]:
# Criar pastas se não existirem
os.makedirs('../../models', exist_ok=True)
os.makedirs('../../data', exist_ok=True)
print("✅ Diretórios criados/verificados")

## 5.2. Salvar Modelo

In [ ]:
# Salvar modelo
model_path = '../../models/baseline_model.pkl'
joblib.dump(baseline_model, model_path)

print(f"✅ Modelo salvo: {model_path}")
print(f"   Tamanho: {os.path.getsize(model_path)/1024:.2f} KB")

# Verificar salvamento
loaded_model = joblib.load(model_path)
print("✅ Verificado! Modelo pode ser carregado")

## 5.3. Salvar Conjuntos de Dados

In [ ]:
# Salvar splits
X_train.to_csv('../../data/X_train.csv', index=False)
X_val.to_csv('../../data/X_val.csv', index=False)
X_test.to_csv('../../data/X_test.csv', index=False)

y_train.to_csv('../../data/y_train.csv', index=False)
y_val.to_csv('../../data/y_val.csv', index=False)
y_test.to_csv('../../data/y_test.csv', index=False)

print("✅ Datasets salvos:")
print("  - X_train.csv, X_val.csv, X_test.csv")
print("  - y_train.csv, y_val.csv, y_test.csv")

## 5.4. Salvar Métricas

In [ ]:
# Salvar métricas para referência
metrics_summary = pd.DataFrame({
    'Treino': train_metrics,
    'Validação': val_metrics
})

metrics_summary.to_csv('../../data/baseline_metrics.csv')
print("✅ Métricas salvas: baseline_metrics.csv")

---
# RESUMO FINAL

In [ ]:
print("="*70)
print("  🎯 ETAPA 3 CONCLUÍDA - MODELO BASELINE")
print("="*70)

print("\n📊 DIVISÃO DE DADOS:")
print(f"  Treino:    {len(X_train):4d} ({len(X_train)/len(X)*100:.1f}%)")
print(f"  Validação: {len(X_val):4d} ({len(X_val)/len(X)*100:.1f}%)")
print(f"  Teste:     {len(X_test):4d} ({len(X_test)/len(X)*100:.1f}%)")

print("\n📈 PERFORMANCE (Validação):")
print(f"  R²:   {val_metrics['R2']:.4f} ({val_metrics['R2']*100:.2f}% variância explicada)")
print(f"  RMSE: {val_metrics['RMSE']:.4f} pontos")
print(f"  MAE:  {val_metrics['MAE']:.4f} pontos")

print("\n💾 ARQUIVOS SALVOS:")
print("  - Modelo: baseline_model.pkl")
print("  - Datasets: X_train, X_val, X_test")
print("  - Métricas: baseline_metrics.csv")

print("\n✅ CHECKLIST:")
print("  ✅ Dados divididos (60/20/20)")
print("  ✅ Modelo baseline treinado")
print("  ✅ Métricas calculadas (MSE, RMSE, MAE, R²)")
print("  ✅ Resíduos analisados")
print("  ✅ Feature importance identificada")
print("  ✅ Overfitting/Underfitting diagnosticado")
print("  ✅ Modelo e dados salvos")

print("\n🚀 PRÓXIMA ETAPA:")
print("  → Etapa 4: Modelos Avançados")
print("  → Meta: Superar R² do baseline em 5-10%")
print("  → Modelos: Random Forest, XGBoost, etc.")

print("\n" + "="*70)
print("  🎓 PARABÉNS! Baseline estabelecido com sucesso!")
print("="*70)